### Optical flow-based tracking

In [1]:
import cv2
import numpy as np

In [4]:
def start_tracking():
    #Define a function to track the object
    cap = cv2.VideoCapture('data/startup.mp4')   #Initialize the video capture object
    scaling_factor = 0.5   #Define the scaling factor for the frames
    
    #Number of frames to track
    num_frames_to_track = 5
    
    #Skipping factor
    num_frames_jump = 2
    
    #Initialize variables
    tracking_paths = []
    frame_index = 0
    
    #Define tracking parameters
    tracking_params = dict(winSize = (11, 11), maxLevel = 2, 
                           criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    
    #Iterate until the user hits the 'Esc' key
    while True:
        #Capture the current frame
        _, frame = cap.read()
        
        #Resize the frame
        frame = cv2.resize(frame, None, fx = scaling_factor, fy = scaling_factor, interpolation = cv2.INTER_AREA)
        
        #Convert the frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        #Create a copy of the frame
        output_img = frame.copy()
        
        #Get images if tracking path remaining is more than 0
        if len(tracking_paths) > 0:
            prev_img, current_img = prev_gray, frame_gray
        
            #Organize the feature points
            feature_points_0 = np.float32([tp[-1] for tp in tracking_paths]).reshape(-1, 1, 2)
        
            #Compute the optical flow 
            feature_points_1, _, _ = cv2.calcOpticalFlowPyrLK(prev_img, current_img, feature_points_0, None, **tracking_params)
        
            #Compute reverse optical flow
            feature_points_0_rev, _, _ = cv2.calcOpticalFlowPyrLK(current_img, prev_img, feature_points_1, None, 
                                                                  **tracking_params)
        
            #Compute the difference between the forward and reverse optical flow
            diff_feature_points = abs(feature_points_0 - feature_points_0_rev).reshape(-1, 2).max(-1)
        
            #Extract the good points
            good_points = diff_feature_points < 1
        
            #Initialize variable
            new_tracking_paths = []
        
            #Iterate through all the good feature points and draw circles around them
            for tp, (x, y), good_points_flag in zip(tracking_paths, feature_points_1.reshape(-1, 2), good_points):
                #If the flag is not true, then continue
                if not good_points_flag:
                    continue
                
                #Append the X and Y coordinates and check if the length is greater than the threshold
                tp.append((x, y))
                if len(tp) > num_frames_to_track:
                    del tp[0]
                
                new_tracking_paths.append(tp)
            
                #Draw a circle around the feature points
                cv2.circle(output_img, (int(x), int(y)), 3, (0, 255, 0), -1)
            
            #Update the tracking paths
            tracking_paths = new_tracking_paths
        
            #Draw lines using the new tracking paths to show movement
            cv2.polylines(output_img, [np.int32(tp) for tp in tracking_paths], False, (0, 150, 0))
            
        #Go into this after skipping the right number of frames
        if not frame_index % num_frames_jump:
            #Create a mask and draw the circles
            mask = np.zeros_like(frame_gray)
            mask[:] = 255
            for x, y in [np.int32(tp[-1]) for tp in tracking_paths]:
                cv2.circle(mask, (int(x), int(y)), 6, 0, -1)
                
            #Compute good features to track
            feature_points = cv2.goodFeaturesToTrack(frame_gray, mask = mask, maxCorners = 500, qualityLevel = 0.3,
                                                    minDistance = 7, blockSize = 7)
            #If feature points exist, append them to tracking paths
            if feature_points is not None:
                for x, y in np.float32(feature_points).reshape(-1, 2):
                    tracking_paths.append([(x, y)])
                    
        #Update variables
        frame_index += 1
        prev_gray = frame_gray
        
        #Display output
        cv2.imshow('Optical Flow', output_img)
        
        #Check if the user hits the 'Esc' key to exit
        stop_key = cv2.waitKey(1)
        if stop_key == 27:
            break

In [ ]:
start_tracking()
cv2.destroyAllWindows()

### Using Haar cascades for object detection

In [3]:
#Load the Haar cascade file corresponding to face detection
face_cascade = cv2.CascadeClassifier('data/haar_cascade_files/haarcascade_frontalface_default.xml')

#Check if the file has been loaded correctly
if face_cascade.empty():
    raise IOError('Unable to load the face cascade classifier xml file')

In [11]:
#Initialize the video capture object
cap = cv2.VideoCapture('data/rodney_darkoo_dance.mp4')

#Define the scaling factor
scaling_factor = 0.5

#Iterate until the user hits the 'Esc' key
while True:
    #Capture the current frame
    _, frame = cap.read()
    
    #Resize the frame
    frame = cv2.resize(frame, None, fx = scaling_factor, fy = scaling_factor, interpolation = cv2.INTER_AREA)
    
    #Convert the image to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #Run the face detector on the grayscale image
    face_rects = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    #Draw a rectangle around the captured face
    for (x, y, w, h) in face_rects:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
        
    #Display the output
    cv2.imshow('Face Detector', frame)
    
    #Check if User hits the 'Esc' key
    stop_key = cv2.waitKey(1)
    if stop_key == 27:
        break
        
#Release the video capture object
cap.release()

#Close all windows
cv2.destroyAllWindows()

### Eye detection and tracking

In [12]:
#Load the Haar cascade files for face and eye
face_cascade = cv2.CascadeClassifier('data/haar_cascade_files/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('data/haar_cascade_files/haarcascade_eye.xml')

#Check if the files were loaded correctly
if face_cascade.empty():
    raise IOError('Unable to load the face cascade classifier xml file')
elif eye_cascade.empty():
    raise IOError('Unable to load the eye cascade classifier xml file')

In [26]:
#Initialize the video capture object
cap = cv2.VideoCapture('data/charlies_angels.mp4')

#Define the scaling factor
scaling_factor = 0.5

#Iterate until User hits the 'Esc' key
while True:
    #Capture the current frame
    _, frame = cap.read()
    
    #Resize the frame
    frame = cv2.resize(frame, None, fx = scaling_factor, fy = scaling_factor, interpolation = cv2.INTER_AREA)
    
    #Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #Run the face detector on the grayscale image
    faces_rect = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    #For each face that's detected, run the eye detector
    for (x, y, w, h) in faces_rect:
        #Extract the grayscale face region of interest (ROI)
        roi_gray = gray[y:y+h, x:x+w]
        
        #Extract the color face ROI for future use
        roi_color = frame[y:y+h, x:x+w]
        
        #Run the eye detector on the grayscale ROI
        eyes_rect = eye_cascade.detectMultiScale(roi_gray)
        
        #Draw circles around the found eyes
        for (x_eye, y_eye, w_eye, h_eye) in eyes_rect:
            center = (int(x_eye + 0.5 * w_eye), int(y_eye + 0.5 * h_eye))
            radius = int(0.3 * (w_eye + h_eye))
            
            color = (0, 255, 0)
            thickness = 3
            
            cv2.circle(roi_color, center, radius, color, thickness)
            
    #Display the output
    cv2.imshow('Eye detector', frame)
    
    #Check if the user hits the 'Esc' key
    stop_key = cv2.waitKey(1)
    if stop_key == 27:
        break
        
#Release the videocapture object
cap.release()

#Close all the windows
cv2.destroyAllWindows()